# Embeddings
https://platform.openai.com/docs/models#embeddings

임베딩(Embeddings)은 텍스트를 수치적으로 표현한 값으로, 두 텍스트 간의 연관성을 측정하는 데 사용된다.

임베딩은 검색, 군집화(clustering), 추천 시스템, 이상 탐지, 분류와 같은 작업에 유용하다.

**모델 및 출력 차원**

| 모델 이름                     | 설명                                                              | 출력 차원 |
|-------------------------------|-------------------------------------------------------------------|-----------|
| **text-embedding-3-large**   | 영어 및 비영어 작업 모두에서 가장 강력한 성능을 가진 모델           | 3,072     |
| **text-embedding-3-small**   | 2세대 ada 임베딩 모델보다 성능이 향상된 모델                        | 1,536     |
| **text-embedding-ada-002**   | 1세대 모델 16개를 대체하는 가장 강력한 2세대 임베딩 모델             | 1,536     |


## MTEB Leaderboard
**Massive Text Embedding Benchmark (MTEB) Leaderboard**

https://huggingface.co/spaces/mteb/leaderboard

**MTEB Leaderboard**는 Hugging Face에서 제공하는 벤치마크 리더보드 페이지로, 다양한 언어 모델(Language Model)과 임베딩 모델(Embedding Model)의 성능을 객관적으로 비교·평가하는 공간이다.

**MTEB Leaderboard에서 순위 산정 방식**

**MTEB Leaderboard**의 순위는 다양한 자연어 처리 태스크(분류, 클러스터링, 검색, 문장 유사도 등)에서 모델이 얻은 점수들의 평균을 기반으로 산정된다. 즉, 여러 벤치마크 데이터셋에서 모델의 성능을 측정하고, 이를 종합하여 평균 점수를 계산한 뒤, 이 평균 점수가 높은 순서대로 모델이 정렬된다.

**주요 평가 방식**

- **평가 태스크 종류**
  - 분류(Classification): F1 점수
  - 클러스터링(Clustering): V-measure
  - 쌍 분류(Pair Classification): Average Precision
  - 재정렬(Reranking): MRR@k, MAP
  - 검색(Retrieval): nDCG@k
  - 의미 유사도(STS): Spearman correlation
  - 요약(Summarization): Spearman correlation  
  각 태스크별로 대표적인 평가 지표가 다르며, 모델은 여러 태스크에서 평가를 받는다[2].

- **평균 점수 산정**
  - 각 태스크별로 모델이 얻은 점수를 모두 합산한 뒤, 태스크 수로 나누어 평균 점수를 구한다.
  - 이 평균 점수가 리더보드의 기본 순위 기준이 된다.

- **부분 평가 가능**
  - 모든 태스크를 수행하지 않아도 특정 태스크만 평가받아 부분 리더보드에 오를 수 있다. 예를 들어, 클러스터링 태스크만 평가받아 해당 부분 순위에 표시될 수 있다.




In [2]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [5]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

text = '안녕하세요~'

response = client.embeddings.create(
    model='text-embedding-3-small',
    input=[text]
)
print(response)

print(len(response.data[0].embedding))

CreateEmbeddingResponse(data=[Embedding(embedding=[0.005405259784311056, -0.07608472555875778, -0.0010263327276334167, 0.021787697449326515, 0.039931148290634155, -0.03833123669028282, -0.04813070222735405, 0.03877565637230873, -0.04848623648285866, -0.030664989724755287, -0.05355262756347656, 0.007449592463672161, 0.013665919192135334, 0.003510919166728854, -0.03497586399316788, 0.027242954820394516, -0.0457308329641819, 0.01664353348314762, -0.024220896884799004, 0.020243337377905846, 0.06124109402298927, 0.010810519568622112, -0.011888238601386547, -0.01635466143488884, 0.04035335034132004, 0.05444146692752838, 0.05644135922193527, -0.027042966336011887, 0.04401981458067894, -0.04755295440554619, 0.027776258066296577, -0.038220133632421494, -0.015554704703390598, -0.04897509887814522, -0.024443108588457108, 0.003510919166728854, 0.017876800149679184, -0.02546527422964573, -0.027687374502420425, -0.03750906139612198, -0.01471030618995428, -0.009460593573749065, -0.0029220625292509794

In [7]:
def texts_to_embedding(texts, model='text-embedding-3-small'):
    # 특수문자/개행문자등을 제거하면 임베딩품질 높아진다.
    texts = [text.replace('\n', ' ') for text in texts]
    response = client.embeddings.create(
        model=model,
        input=texts
    )
    return [data.embedding for data in response.data]

output = texts_to_embedding(['hello world', 'byebye world'])

import numpy as np
np.array(output).shape

(2, 1536)

## 음식리뷰 유사도 검색

In [20]:
!gdown 1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD

Downloading...
From: https://drive.google.com/uc?id=1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD
To: /content/fine_food_reviews_1k.csv
100% 439k/439k [00:00<00:00, 23.7MB/s]


In [22]:
# 데이터로드
import pandas as pd

df = pd.read_csv('fine_food_reviews_1k.csv', index_col=0)
df.head()

,Time,ProductId,UserId,Score,Summary,Text,n_tokens
Unnamed: 0,,,,,,,
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,33
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",26
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,242
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,216
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,76


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Time       1000 non-null   int64 
 1   ProductId  1000 non-null   object
 2   UserId     1000 non-null   object
 3   Score      1000 non-null   int64 
 4   Summary    1000 non-null   object
 5   Text       1000 non-null   object
 6   n_tokens   1000 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 62.5+ KB


In [24]:
# Summary + Text -> Combined
df['combined'] = df['Summary'].str.strip() + '; ' + df['Text'].str.strip()
df.head()

,Time,ProductId,UserId,Score,Summary,Text,n_tokens,combined
Unnamed: 0,,,,,,,,
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,33,where does one start...and stop... with a tre...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",26,Arrived in pieces; Not pleased at all. When I ...
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,242,"It isn't blanc mange, but isn't bad . . .; I'm..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,216,These also have SALT and it's not sea salt.; I...
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,76,Happy with the product; My dog was suffering w...


In [25]:
# 임베딩변환
df['embedding'] = texts_to_embedding(df['combined'].tolist())
df.head()

,Time,ProductId,UserId,Score,Summary,Text,n_tokens,combined,embedding
Unnamed: 0,,,,,,,,,
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,33,where does one start...and stop... with a tre...,"[0.030276387929916382, -0.020651785656809807, ..."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",26,Arrived in pieces; Not pleased at all. When I ...,"[0.01129516027867794, 0.03488067910075188, -0...."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,242,"It isn't blanc mange, but isn't bad . . .; I'm...","[0.0022991024889051914, 0.004840383306145668, ..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,216,These also have SALT and it's not sea salt.; I...,"[-0.015768831595778465, 0.013766037300229073, ..."
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,76,Happy with the product; My dog was suffering w...,"[7.225894660223275e-05, -0.06748031824827194, ..."


In [32]:
df['embedding'].tolist()

Output hidden; open in https://colab.research.google.com to view.

In [37]:
# embed_df로 변환
embed_df = df[['embedding']]
# embed_df = df['embedding'].to_frame()
embed_df.index = df['combined']
embed_df.head()

,embedding
combined,
where does one start...and stop... with a treat like this; Wanted to save some to bring to my Chicago family but my North Carolina family ate all 4 boxes before I could pack. These are excellent...could serve to anyone,"[0.030276387929916382, -0.020651785656809807, ..."
"Arrived in pieces; Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.","[0.01129516027867794, 0.03488067910075188, -0...."
"It isn't blanc mange, but isn't bad . . .; I'm not sure that custard is really custard without eggs. But this comes close. I got it for use in a ""Vegan pancake"" recipe. We were having houseguests who were Vegan and I wanted to make some special breakfasts while they were here. One of the cooking/recipe sites had a recipe using this and there were lots of great reviews. I tried the recipe and it turned out like wallpaper paste -- yuck!<br />However, the so-called custard isn't so bad. I think it's probably just cornstarch and annatto (yellow coloring with a slight flavor). It's fun playing with it. You could dress it up with fruit. Seems to come out on the thin side when you make it as directed, so I use less milk because I like my custards to set firm. As a custard sauce it's fine. I would say it tastes something between a pudding and a custard.<br /><br />If you want a really good egg-free ""custard"" get an original recipe for ""blanc mange."" It takes a lot longer to make, but it's certainly worth the difference.","[0.0022991024889051914, 0.004840383306145668, ..."
"These also have SALT and it's not sea salt.; I like the fact that you can see what you're getting and that there are no bones or dark meat. There are 7 nice big chunks in every jar.<br /><br />These taste like tuna in a can but, because they're preserved in glass, you don't have to worry about either aluminum or BPA; BUT ... they are not just tuna and spring water.<br /><br />There is salt in there, too, and it's not healthy sea salt, it's toxic table salt.<br /><br />I am trying to contact Tonnino to confirm that. I might be wrong because the label states that the ingredients are ""tuna fish"" but the sticker on the top clarifies that it is the smaller (healthier) yellowfin, so the ""salt"" listed in the ingredients might be sea salt but, if it was, why don't they say so?<br /><br />Without confirmation, I will continue to look for a salt-free olive-oil free tuna preserved in glass.<br /><br />If you know of one, please contact me!","[-0.015768831595778465, 0.013766037300229073, ..."
Happy with the product; My dog was suffering with itchy skin. He had been eating Natural Choice brand (cheaper) since he was a puppy. I was nervous to change foods. The vet suggested to change foods sand see if the skin issues cleared up. Wellness brand did the job. My dog seems to love the food and the skin issues cleared up within a few weeks.,"[7.225894660223275e-05, -0.06748031824827194, ..."


In [51]:
# 유사도 검색
from sklearn.metrics.pairwise import cosine_similarity

def review_search(query, emb_df, top_n=5):
    # 검색어 동일한 임베딩변환
    query_emb = texts_to_embedding([query])

    embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])
    top_n_texts = embed_df.sort_values('cos_sim', ascending=False).head(top_n)
    return top_n_texts

review_search('delicious fruit', embed_df)

,embedding,cos_sim
combined,,
"Delicious .; These plums are sweet and juicy, and the aroma is like perfume. And it doesn't hurt that they are good for you, too.","[0.03155634179711342, -0.0019496827153488994, ...",0.615416
"Delicious!; For anyone who says ""I don't like fruitcake"" or anyone who's never had fruitcake and wonders what all the fuss is about, try this. (As long as you're not allergic to tree nuts or any other ingredient.) It's chock-a-block with nuts and moist fruit. I will definitely be buying more.","[-0.016479600220918655, 0.0018902381416410208,...",0.602955
"These are Delicious!; Great taste, right price, fabulous snack! It is the only fruit I can get my little one to eat and I can't keep my high school son out of them either. They are great pick-me-ups on the way to my daughter's soccer practice or before my early morning run. And best of all, Amazon ships these right to my door every month. No more finding the right store who carries them. I just set up the automatic recurring shipment once and it works like a charm!","[0.028737444430589676, -0.01361179817467928, -...",0.583637
"Delicious!; Wonderful! Deep, rich, pure black raspberry syrup! Absolutely delicious on waffles, cheesecake, ice cream, yogurt, drinks, etc. Thrilled to see that there are at least some berry syrup makers who do not feel the need to ""sour"" the flavor of perfect berry products with citric acid!","[0.008850290440022945, -0.037961218506097794, ...",0.536565
"What a treat!; Ordered these as part of a presentation on Malaysia as this fruit had left a very positive impression on me during my visit. We gave these to 4th graders who were equally impressed with the exterior soft spikes and the juicy center. The fruits arrived in good condition, just slightly bruised. The kept well in the fridge in a single-double layer covered with a damp paper towel. I had also rinsed in a vinegar-water (1:10 parts) solution to be sure to inhibit any mold (like I do with berries). They kept from Saturday when they arrived to Thursday with minimal browning. I ordered 4 pounds and had plenty of fruits for the 40 kids. I think there were about 55 - 60 fruits all together. It was a fantastic treat and the kids were requesting we bring them again for the Halloween party! I'm sure they'll remember the presentation for a long time.","[-0.025338858366012573, -0.0016857946757227182...",0.485380


In [55]:
review_search('bad delievery', embed_df)

,embedding,cos_sim
combined,,
"great product, poor delivery; The coffee is excellent and I am a repeat buyer. Problem this time was with the UPS delivery. They left the box in front of my garage door in the middle of the driveway. Because of this odd delivery location, my wife ran over the box when she backed out of the garage and did not see the box. we lost half of the cups. Thius is the third time I have written about this matter to Amazon with no results. Hopefully someone will respond to me.<br /><br />fred santaniello","[-0.032880187034606934, -0.010327393189072609,...",0.462548
"great product, poor delivery; The coffee is excellent and I am a repeat buyer. Problem this time was with the UPS delivery. They left the box in front of my garage door in the middle of the driveway. Because of this odd delivery location, my wife ran over the box when she backed out of the garage and did not see the box. we lost half of the cups. Thius is the third time I have written about this matter to Amazon with no results. Hopefully someone will respond to me.<br /><br />fred santaniello","[-0.032881032675504684, -0.010334637016057968,...",0.462486
"great product, poor delivery; The coffee is excellent and I am a repeat buyer. Problem this time was with the UPS delivery. They left the box in front of my garage door in the middle of the driveway. Because of this odd delivery location, my wife ran over the box when she backed out of the garage and did not see the box. we lost half of the cups. Thius is the third time I have written about this matter to Amazon with no results. Hopefully someone will respond to me.<br /><br />fred santaniello","[-0.032881032675504684, -0.010334637016057968,...",0.462486
"great product, poor delivery; The coffee is excellent and I am a repeat buyer. Problem this time was with the UPS delivery. They left the box in front of my garage door in the middle of the driveway. Because of this odd delivery location, my wife ran over the box when she backed out of the garage and did not see the box. we lost half of the cups. Thius is the third time I have written about this matter to Amazon with no results. Hopefully someone will respond to me.<br /><br />fred santaniello","[-0.032881032675504684, -0.010334637016057968,...",0.462486
"The Seller is NOT Starbursts; As I ordered this for my boyfriend for his birthday as he LOVES the sour Starbursts but they were discontinued. So I ordered these and found that not only is the seller a scammer, I never received my product!<br /><br />After multiple attempts and emails to ARRAM, I had to file a claim. They said they would ""redeliver"" this product THREE times and I still didn't receive it. They write their own feedback for a higher score and never solved my problem.<br /><br />DO NOT ORDER THESE! You will never receive them. Very disappointed that I was scammed on Amazon. I never write reviews however, I wanted to ensure that this does not happen to anyone else.","[-0.004279153887182474, -0.020588990300893784,...",0.388735
